# Lezione 3 — GroupBy e Transform: Statistiche di Gruppo

---

## Obiettivi della Lezione

Al termine di questa lezione sarai in grado di:

1. Usare **`.groupby()`** per raggruppare dati per categoria
2. Applicare **`.transform()`** per ottenere statistiche di gruppo riga-per-riga
3. Capire la differenza tra **`.agg()`** e **`.transform()`**
4. Creare colonne che mostrano **differenze rispetto alla media di gruppo**
5. Interpretare i **delta** (scostamenti) per analisi comparative
6. Ordinare i risultati per facilitare l'interpretazione

---

## Prerequisiti

- Lezione 2: Filtri e Feature Engineering con Pandas
- Conoscenza di DataFrame, Series e operazioni vettoriali

---

## Indice

1. **SEZIONE 1** — Teoria Concettuale Approfondita
2. **SEZIONE 2** — Schema Mentale / Mappa Decisionale
3. **SEZIONE 3** — Notebook Dimostrativo
4. **SEZIONE 4** — Metodi Spiegati
5. **SEZIONE 5** — Glossario
6. **SEZIONE 6** — Errori Comuni e Debug Rapido
7. **SEZIONE 7** — Conclusione Operativa
8. **SEZIONE 8** — Checklist di Fine Lezione
9. **SEZIONE 9** — Changelog Didattico

---

## Librerie Utilizzate

```python
import pandas as pd
```

---

# SEZIONE 1 — Teoria Concettuale Approfondita

## 1.1 Il Paradigma Split-Apply-Combine

**GroupBy** implementa il paradigma **Split-Apply-Combine**:

```
    DATI ORIGINALI
          |
          v
    ┌─────────────┐
    │    SPLIT    │  Dividi per gruppi
    └─────────────┘
          |
    ┌─────┴─────┐
    v           v
  Gruppo A   Gruppo B
    |           |
    v           v
    ┌─────────────┐
    │    APPLY    │  Applica funzione a ogni gruppo
    └─────────────┘
    |           |
    v           v
  Result A   Result B
    |           |
    └─────┬─────┘
          v
    ┌─────────────┐
    │   COMBINE   │  Unisci i risultati
    └─────────────┘
          |
          v
      OUTPUT
```

## 1.2 La Differenza Cruciale: `.agg()` vs `.transform()`

Questa e la distinzione piu importante da capire:

| Aspetto | `.agg()` / `.mean()` | `.transform()` |
|---------|---------------------|----------------|
| **Output** | Una riga per gruppo | Stesse righe dell'originale |
| **Lunghezza** | Ridotta | Invariata |
| **Uso tipico** | Report, statistiche riassuntive | Aggiungere stat a ogni riga |
| **Indice** | Gruppi come indice | Indice originale |

**Esempio visivo:**

```
DataFrame originale (7 righe):
   Pioggia  Temp
0    0      18
1    1      20
2    0      21
3    1      19
4    0      23
5    0      25
6    1      22

.groupby("Pioggia")["Temp"].mean()     .groupby("Pioggia")["Temp"].transform("mean")
→ 2 righe (una per gruppo)              → 7 righe (stessa lunghezza)

Pioggia                                  0    21.75  (media gruppo 0)
0    21.75                               1    20.33  (media gruppo 1)
1    20.33                               2    21.75
                                         3    20.33
                                         4    21.75
                                         5    21.75
                                         6    20.33
```

## 1.3 Perche Transform e Fondamentale?

Con `transform()` puoi:

1. **Confrontare ogni riga con la media del suo gruppo**
   ```python
   df["Delta"] = df["Valore"] - df.groupby("Gruppo")["Valore"].transform("mean")
   ```

2. **Normalizzare per gruppo (z-score di gruppo)**
   ```python
   df["Z_Gruppo"] = (df["Val"] - df.groupby("G")["Val"].transform("mean")) / \
                     df.groupby("G")["Val"].transform("std")
   ```

3. **Calcolare percentuali sul totale di gruppo**
   ```python
   df["Pct_Gruppo"] = df["Val"] / df.groupby("G")["Val"].transform("sum") * 100
   ```

## 1.4 Interpretazione del Delta (Scostamento)

Il **delta** (differenza dalla media) e una metrica potente:

$$\Delta_i = x_i - \bar{x}_{gruppo}$$

| Valore Delta | Interpretazione |
|--------------|-----------------|
| $\Delta > 0$ | Sopra la media del gruppo |
| $\Delta < 0$ | Sotto la media del gruppo |
| $\Delta \approx 0$ | In linea con la media |
| $|\Delta|$ grande | Outlier potenziale nel gruppo |

## 1.5 Vantaggi di Transform rispetto ai Join

Senza `transform()`, dovresti:
1. Calcolare le medie con `.agg()`
2. Fare un merge/join per riunire al DataFrame originale

Con `transform()`:
- Un solo passaggio
- Indici gia allineati
- Codice piu pulito e leggibile

In [ ]:
# === SETUP: Import libreria e creazione DataFrame ===

import pandas as pd  # Libreria per manipolazione dati tabulari

# Creiamo un DataFrame con dati meteorologici settimanali
df = pd.DataFrame({
    "Giorno": ["Lun", "Mar", "Mer", "Gio", "Ven", "Sab", "Dom"],
    "Temperatura": [18, 20, 21, 19, 23, 25, 22],  # Gradi Celsius
    "Umidita": [60, 55, 58, 63, 50, 45, 52],      # Percentuale
    "Pioggia": [0, 1, 0, 1, 0, 0, 1],             # 0=No, 1=Si (variabile di gruppo)
    "Vento": [5, 12, 8, 20, 7, 6, 15]             # km/h
})

# Visualizziamo il DataFrame
print("DataFrame meteo creato!")
print(f"Dimensioni: {df.shape[0]} righe x {df.shape[1]} colonne")
print(f"Giorni con pioggia: {df['Pioggia'].sum()}")
print(f"Giorni senza pioggia: {(df['Pioggia'] == 0).sum()}")
print()
df

---

# SEZIONE 2 — Schema Mentale / Mappa Decisionale

## Quando Usare GroupBy?

```
DEVO ANALIZZARE DATI PER CATEGORIE?
                |
                v
             SI/NO
            /      \
          SI        NO
           |         |
           v         v
    .groupby()    Operazioni
                  su tutto df
```

## Quale Metodo Dopo GroupBy?

```
DOPO .groupby(), COSA MI SERVE?
                |
                v
    ┌───────────┴───────────┐
    |                       |
    v                       v
STATISTICHE             STATISTICHE
RIASSUNTIVE            PER OGNI RIGA
(1 riga/gruppo)        (stesse righe)
    |                       |
    v                       v
.agg() / .mean()      .transform()
.sum() / .count()
```

## Pattern Comune: Calcolo Delta

```
1. Raggruppa per categoria
         |
         v
2. Calcola media con transform()
         |
         v
3. Sottrai: valore - media_gruppo
         |
         v
4. Ottieni DELTA (scostamento)
         |
         v
5. Ordina per delta (analisi)
```

## Checklist Pre-GroupBy

1. Quale colonna definisce i gruppi?
2. Quale colonna voglio aggregare?
3. Mi serve output ridotto o espanso?
4. Devo confrontare righe con statistiche di gruppo?

---

# SEZIONE 3 — Notebook Dimostrativo

## 3.1 Setup: Import e Creazione DataFrame

**Perche questo passaggio:** Creiamo un dataset meteo settimanale per esplorare groupby e transform. La colonna `Pioggia` (0/1) sara la nostra variabile di raggruppamento.

---

## 3.2 GroupBy con Transform: Media per Gruppo

**Perche questo passaggio:** Vogliamo calcolare la temperatura media per ogni gruppo di pioggia (0 e 1), ma mantenendo la stessa lunghezza del DataFrame originale.

**Concetto chiave:**
- `.groupby("Pioggia")` separa le righe in 2 gruppi
- `["Temperatura"]` seleziona la colonna da aggregare
- `.transform("mean")` calcola la media MA restituisce 7 valori (non 2)

In [ ]:
# === TRANSFORM: Media temperatura per gruppo di pioggia ===

# Raggruppiamo per Pioggia e calcoliamo la media della Temperatura
# transform() restituisce una Series con la stessa lunghezza del df originale
temp_media_gruppo = df.groupby("Pioggia")["Temperatura"].transform("mean")

# Mostriamo il risultato
print("Media temperatura per gruppo (via transform):")
print(temp_media_gruppo)
print()

# --- MICRO-CHECKPOINT ---
# Verifichiamo che la lunghezza sia uguale al DataFrame originale
assert len(temp_media_gruppo) == len(df), "Transform deve mantenere la lunghezza!"

# Verifichiamo che ci siano esattamente 2 valori unici (uno per gruppo)
n_valori_unici = temp_media_gruppo.nunique()
print(f"--- Micro-checkpoint: {len(temp_media_gruppo)} valori, {n_valori_unici} unici (2 gruppi) ---")

# Confronto con .mean() classico
print("\nConfronto con .mean() (riduce a 2 righe):")
print(df.groupby("Pioggia")["Temperatura"].mean())

,Giorno,Temperatura,Umidita,Pioggia,Vento,Temp_Delta_Pioggia
0,Lun,18,60,0,5,-3.750000
1,Mar,20,55,1,12,-0.333333
2,Mer,21,58,0,8,NaN
3,Gio,19,63,1,20,NaN
4,Ven,23,50,0,7,NaN
5,Sab,25,45,0,6,NaN
6,Dom,22,52,1,15,NaN


---

## 3.3 Calcolo del Delta: Scostamento dalla Media di Gruppo

**Perche questo passaggio:** Creiamo una nuova colonna che mostra quanto ogni giorno si discosta dalla media del suo gruppo. Questo e il cuore dell'analisi comparativa intra-gruppo.

**Formula:**
$$\Delta_i = \text{Temperatura}_i - \bar{\text{Temperatura}}_{gruppo}$$

In [ ]:
# === CALCOLO DELTA: Scostamento dalla media di gruppo ===

# Creiamo la colonna con la media del gruppo per ogni riga
df["Temp_Media_Gruppo"] = temp_media_gruppo

# Calcoliamo il delta: quanto ogni giorno si discosta dalla media del suo gruppo
df["Temp_Delta_Pioggia"] = df["Temperatura"] - df["Temp_Media_Gruppo"]

# --- MICRO-CHECKPOINT ---
# La somma dei delta per gruppo deve essere circa 0 (proprieta della media)
delta_sum_gruppo0 = df.loc[df["Pioggia"] == 0, "Temp_Delta_Pioggia"].sum()
delta_sum_gruppo1 = df.loc[df["Pioggia"] == 1, "Temp_Delta_Pioggia"].sum()

print(f"Somma delta gruppo Pioggia=0: {delta_sum_gruppo0:.6f} (deve essere ~0)")
print(f"Somma delta gruppo Pioggia=1: {delta_sum_gruppo1:.6f} (deve essere ~0)")

assert abs(delta_sum_gruppo0) < 0.0001, "Errore nel calcolo delta!"
assert abs(delta_sum_gruppo1) < 0.0001, "Errore nel calcolo delta!"
print("\n--- Micro-checkpoint: somme delta = 0 (corretto!) ---")

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,Temp_Delta_Pioggia
0,Lun,18,60,0,5,-3.750000
1,Mar,20,55,1,12,-0.333333
2,Mer,21,58,0,8,-0.750000
3,Gio,19,63,1,20,-1.333333
4,Ven,23,50,0,7,1.250000
5,Sab,25,45,0,6,3.250000
6,Dom,22,52,1,15,1.666667


---

## 3.4 Ordinamento per Interpretazione

**Perche questo passaggio:** Ordiniamo per delta decrescente per vedere prima i giorni piu caldi rispetto alla media del loro gruppo. Questo facilita l'identificazione di anomalie.

**Interpretazione:**
- Delta > 0: giorno piu caldo della media del suo gruppo
- Delta < 0: giorno piu freddo della media del suo gruppo

In [ ]:
# === ORDINAMENTO PER DELTA (decrescente) ===

# Ordiniamo per vedere prima i giorni con maggiore scostamento positivo
df_ordinato = df.sort_values("Temp_Delta_Pioggia", ascending=False)

# --- MICRO-CHECKPOINT ---
# Il primo valore deve essere il piu alto, l'ultimo il piu basso
primo_delta = df_ordinato["Temp_Delta_Pioggia"].iloc[0]
ultimo_delta = df_ordinato["Temp_Delta_Pioggia"].iloc[-1]

print(f"Giorno con maggiore delta positivo: {df_ordinato.iloc[0]['Giorno']}")
print(f"  Temperatura: {df_ordinato.iloc[0]['Temperatura']} C")
print(f"  Media gruppo: {df_ordinato.iloc[0]['Temp_Media_Gruppo']:.2f} C")
print(f"  Delta: +{primo_delta:.2f} C")
print()
print(f"Giorno con maggiore delta negativo: {df_ordinato.iloc[-1]['Giorno']}")
print(f"  Temperatura: {df_ordinato.iloc[-1]['Temperatura']} C")
print(f"  Media gruppo: {df_ordinato.iloc[-1]['Temp_Media_Gruppo']:.2f} C")
print(f"  Delta: {ultimo_delta:.2f} C")

assert primo_delta >= ultimo_delta, "Ordinamento non corretto!"
print("\n--- Micro-checkpoint: ordinamento verificato ---")

df_ordinato

,Giorno,Temperatura,Umidita,Pioggia,Vento,Temp_Delta_Pioggia
5,Sab,25,45,0,6,3.250000
6,Dom,22,52,1,15,1.666667
4,Ven,23,50,0,7,1.250000
1,Mar,20,55,1,12,-0.333333
2,Mer,21,58,0,8,-0.750000
3,Gio,19,63,1,20,-1.333333
0,Lun,18,60,0,5,-3.750000


---

# SEZIONE 4 — Metodi Spiegati

## GroupBy

| Metodo | Sintassi | Cosa Fa |
|--------|----------|---------|
| `.groupby()` | `df.groupby("Col")` | Crea oggetto GroupBy |
| Multi-colonna | `df.groupby(["A", "B"])` | Gruppi per combinazioni |
| Selezione | `df.groupby("A")["B"]` | Seleziona colonna da aggregare |

## Aggregazioni (Riducono le Righe)

| Metodo | Sintassi | Output |
|--------|----------|--------|
| `.mean()` | `df.groupby("A")["B"].mean()` | Media per gruppo |
| `.sum()` | `df.groupby("A")["B"].sum()` | Somma per gruppo |
| `.count()` | `df.groupby("A")["B"].count()` | Conteggio per gruppo |
| `.std()` | `df.groupby("A")["B"].std()` | Deviazione standard |
| `.agg()` | `df.groupby("A").agg({"B": "mean"})` | Aggregazioni multiple |

## Transform (Mantiene le Righe)

| Metodo | Sintassi | Output |
|--------|----------|--------|
| `.transform("mean")` | `df.groupby("A")["B"].transform("mean")` | Media gruppo per ogni riga |
| `.transform("sum")` | `df.groupby("A")["B"].transform("sum")` | Somma gruppo per ogni riga |
| `.transform("std")` | `df.groupby("A")["B"].transform("std")` | Std gruppo per ogni riga |
| Lambda | `df.groupby("A")["B"].transform(lambda x: x - x.mean())` | Funzione custom |

## Pattern Comuni con Transform

| Pattern | Codice | Uso |
|---------|--------|-----|
| Delta dalla media | `df["Val"] - df.groupby("G")["Val"].transform("mean")` | Scostamento |
| Percentuale sul totale | `df["Val"] / df.groupby("G")["Val"].transform("sum") * 100` | Quota % |
| Z-score di gruppo | `(df["Val"] - transform("mean")) / transform("std")` | Normalizzazione |

---

# SEZIONE 5 — Glossario

| Termine | Definizione |
|---------|-------------|
| **GroupBy** | Operazione che raggruppa righe per valori di una o piu colonne |
| **Split-Apply-Combine** | Paradigma: dividi per gruppi, applica funzione, combina risultati |
| **Aggregazione** | Riduzione di piu valori a uno solo (media, somma, conteggio) |
| **Transform** | Applicazione di funzione che mantiene la lunghezza originale |
| **Delta** | Differenza tra un valore e un riferimento (es. media di gruppo) |
| **Scostamento** | Sinonimo di delta, misura quanto un valore si discosta dalla media |
| **Intra-gruppo** | All'interno dello stesso gruppo |
| **Inter-gruppo** | Tra gruppi diversi |
| **Statistica di gruppo** | Valore calcolato su tutti gli elementi di un gruppo |
| **Allineamento indici** | Corrispondenza automatica tra indici di Series/DataFrame |
| **Broadcast** | Estensione automatica di valori per operazioni elemento-per-elemento |

---

# SEZIONE 6 — Errori Comuni e Debug Rapido

## Errore 1: Confondere `.mean()` con `.transform("mean")`

```python
# SBAGLIATO - lunghezze diverse!
df["Media"] = df.groupby("Cat")["Val"].mean()  # ValueError!

# CORRETTO - usa transform per mantenere la lunghezza
df["Media"] = df.groupby("Cat")["Val"].transform("mean")
```

## Errore 2: Dimenticare di Selezionare la Colonna

```python
# SBAGLIATO - applica a TUTTE le colonne numeriche
df.groupby("Cat").mean()  # Restituisce DataFrame, non Series

# CORRETTO - seleziona la colonna specifica
df.groupby("Cat")["Val"].mean()
```

## Errore 3: Aspettarsi Modifica In-Place

```python
# ATTENZIONE - groupby restituisce NUOVO oggetto
df.groupby("Cat")["Val"].transform("mean")  # df NON cambia!

# CORRETTO - assegna il risultato
df["Media_Gruppo"] = df.groupby("Cat")["Val"].transform("mean")
```

## Errore 4: Usare Stringa Sbagliata in Transform

```python
# SBAGLIATO - typo nella funzione
df.groupby("Cat")["Val"].transform("mena")  # AttributeError!

# CORRETTO - nomi validi: "mean", "sum", "std", "min", "max", "count"
df.groupby("Cat")["Val"].transform("mean")
```

## Errore 5: GroupBy su Colonna Inesistente

```python
# SBAGLIATO - typo nel nome colonna
df.groupby("Categria")["Val"].mean()  # KeyError!

# CORRETTO - verifica i nomi prima
print(df.columns)  # Controlla i nomi
df.groupby("Categoria")["Val"].mean()
```

## Errore 6: Operazioni Aritmetiche con Lunghezze Diverse

```python
# SBAGLIATO - lunghezze incompatibili
media_per_gruppo = df.groupby("Cat")["Val"].mean()  # 3 righe
df["Delta"] = df["Val"] - media_per_gruppo  # NaN ovunque!

# CORRETTO - usa transform
media_espansa = df.groupby("Cat")["Val"].transform("mean")  # 100 righe
df["Delta"] = df["Val"] - media_espansa  # Funziona!
```

---

# SEZIONE 7 — Conclusione Operativa

## Cosa Hai Imparato

In questa lezione hai acquisito competenze fondamentali per l'analisi comparativa:

1. **GroupBy**: raggruppare dati per categoria
2. **Transform**: ottenere statistiche di gruppo per ogni riga
3. **Differenza agg vs transform**: quando usare quale
4. **Calcolo delta**: scostamento dalla media di gruppo
5. **Interpretazione**: delta > 0 = sopra media, delta < 0 = sotto media
6. **Ordinamento**: per identificare outlier e pattern

## Pattern Ricorrente: Analisi Intra-Gruppo

```python
# 1. Raggruppa e calcola statistica con transform
df["Media_Gruppo"] = df.groupby("Categoria")["Valore"].transform("mean")

# 2. Calcola lo scostamento (delta)
df["Delta"] = df["Valore"] - df["Media_Gruppo"]

# 3. Identifica outlier
df["Sopra_Media"] = df["Delta"] > 0

# 4. Ordina per analisi
df.sort_values("Delta", ascending=False)
```

## Collegamento alla Prossima Lezione

Nella Lezione 4 approfondiremo:
- Categorizzazione di variabili continue
- GroupBy multi-colonna
- Transform per analisi avanzate

---

# SEZIONE 8 — Checklist di Fine Lezione

Prima di procedere alla prossima lezione, verifica di saper fare tutto quanto segue:

- [ ] So usare `.groupby("Colonna")` per raggruppare dati
- [ ] So selezionare una colonna dopo groupby: `.groupby("A")["B"]`
- [ ] Capisco la differenza tra `.mean()` (riduce) e `.transform("mean")` (mantiene)
- [ ] So usare `.transform("mean")` per aggiungere medie di gruppo a ogni riga
- [ ] So calcolare il delta: `valore - media_gruppo`
- [ ] Capisco che la somma dei delta per gruppo e sempre ~0
- [ ] So interpretare delta > 0 come "sopra la media del gruppo"
- [ ] So interpretare delta < 0 come "sotto la media del gruppo"
- [ ] So ordinare per delta per identificare outlier
- [ ] Capisco il paradigma Split-Apply-Combine
- [ ] So che transform mantiene l'allineamento degli indici automaticamente

**Se hai dubbi su qualche punto, rileggi la sezione corrispondente prima di proseguire.**

---

# SEZIONE 9 — Changelog Didattico

| Versione | Data       | Modifiche                                                                 |
|----------|------------|---------------------------------------------------------------------------|
| 1.0      | Originale  | Versione iniziale del notebook                                           |
| 2.0      | 2025-12-30 | Ristrutturazione completa secondo template standard a 9 sezioni          |
|          |            | + Nuovo header con obiettivi, prerequisiti e indice                      |
|          |            | + Aggiunta SEZIONE 1: Teoria (Split-Apply-Combine, agg vs transform)     |
|          |            | + Aggiunta SEZIONE 2: Schema mentale / mappa decisionale                 |
|          |            | + Riorganizzata SEZIONE 3: Notebook dimostrativo con spiegazioni         |
|          |            | + Aggiunti "Perche questo passaggio" prima di ogni cella di codice       |
|          |            | + Aggiunti micro-checkpoint con asserzioni e sanity check                |
|          |            | + Aggiunta colonna Temp_Media_Gruppo per trasparenza                     |
|          |            | + Aggiunta verifica somma delta = 0 (proprieta matematica)               |
|          |            | + Migliorati commenti inline in ogni cella di codice                     |
|          |            | + Aggiunta SEZIONE 4: Metodi spiegati con tabelle di riferimento         |
|          |            | + Aggiunta SEZIONE 5: Glossario (11 termini)                             |
|          |            | + Aggiunta SEZIONE 6: Errori comuni e debug rapido (6 errori)            |
|          |            | + Aggiunta SEZIONE 7: Conclusione operativa                              |
|          |            | + Aggiunta SEZIONE 8: Checklist di fine lezione (11 items)               |
|          |            | + Aggiunta SEZIONE 9: Changelog didattico                                |

---

**Fine della Lezione 3**